In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import gomics
from glob import glob
import itertools
import submitit
import subprocess
from pybedtools import BedTool
import os

In [2]:
LDSC_DATA_DIR = "/u/project/pasaniuc/kangchen/DATA/ldsc/"
DATA_DIR = "../format-dmr-loop/out/"

In [3]:
df_params = {"group": [], "bed": []}

for group_folder in glob(DATA_DIR + "*"):
    group = group_folder.split("/")[-1]
    if "TSS" in group:
        continue
    ct_list = [p.split("/")[-1] for p in glob(os.path.join(DATA_DIR, group, "*.bed*"))]
    df_params["group"].extend([group] * len(ct_list))
    df_params["bed"].extend(ct_list)
df_params = pd.DataFrame(df_params)

df_params["out_dir"] = df_params.apply(
    lambda x: f"out/ldscore/{x.group}/{x.bed.split('.')[0]}", axis=1
)

# filter L2
# df_params = df_params[df_params["group"].str.contains("L2")]
display(df_params.groupby("group").size())

group
L2DMR                  62
L2DMR-L2LOOP           59
L2DMR-L2LOOPSUMMIT     59
L2LOOP                 89
L2LOOP-L2DMR           59
L2LOOPSUMMIT           89
L2LOOPSUMMIT-L2DMR     59
L3DMR                 195
L3DMR-L3LOOP          159
L3DMR-L3LOOPSUMMIT    159
L3LOOP                159
L3LOOP-L3DMR          159
L3LOOPSUMMIT          159
L3LOOPSUMMIT-L3DMR    159
dtype: int64

In [4]:
def make_annot(group, bed, out_dir):
    bed_file = os.path.join(DATA_DIR, group, bed)
    annot_bed = BedTool(bed_file)
    os.makedirs(out_dir, exist_ok=True)
    for chrom in tqdm(range(1, 23)):
        bim_file = os.path.join(
            LDSC_DATA_DIR, "baseline_v1.2", f"baseline.{chrom}.annot.gz"
        )
        df_bim = pd.read_csv(bim_file, delim_whitespace=True)[
            ["CHR", "BP", "SNP", "CM"]
        ]
        df_snp_annot = gomics.make_ldsc_annot(annot_bed, df_bim)
        # in case of duplicated DMR (due to liftOver)
        # already handled in make_ldsc_annot
        # df_snp_annot = df_snp_annot.drop_duplicates(subset="SNP")
        assert np.all(
            df_bim["SNP"].values == df_snp_annot["SNP"].values
        ), f"{group}, {ct}, {out_dir}, {chrom} not consistent"

        # Optionally use 0/1 annotation
        df_snp_annot.loc[df_snp_annot["ANNOT"].notna(), "ANNOT"] = 1
        df_snp_annot.fillna(0, inplace=True)
        print(df_snp_annot["ANNOT"].mean())
        df_snp_annot.to_csv(f"{out_dir}/{chrom}.annot.gz", sep="\t", index=False)

In [5]:
df_todo_params = df_params[
    ~df_params.apply(lambda x: os.path.exists(x.out_dir + "/22.annot.gz"), axis=1)
].reset_index(drop=True)

In [6]:
df_todo_params

,group,bed,out_dir


In [7]:
executor = submitit.SgeExecutor(folder="./submitit-logs")

executor.update_parameters(
    time_min=75,
    memory_g=16,
    setup=[
        "export PATH=~/project-pasaniuc/software/miniconda3/bin:$PATH",
        "export PATH=~/project-pasaniuc/software/bin:$PATH",
        "export PYTHONNOUSERSITE=True",
    ],
)

jobs = executor.map_array(
    make_annot,
    df_todo_params.group,
    df_todo_params.bed,
    df_todo_params.out_dir,
)

/u/project/pasaniuc/kangchen/software/submitit-sge/submitit/core/core.py:699: UserWarning: Received an empty job array
  warnings.warn("Received an empty job array")


In [8]:
def make_ldscore(out_dir):
    LDSC_DIR = "/u/project/pasaniuc/kangchen/software/ldsc"
    PYTHON_PATH = (
        "/u/project/pasaniuc/kangchen/software/miniconda3/envs/ldsc/bin/python"
    )

    for chrom in range(1, 23):
        if os.path.exists(f"{out_dir}/{chrom}.l2.M"):
            continue
        cmds = [
            f"{PYTHON_PATH} {LDSC_DIR}/ldsc.py",
            "--l2",
            f"--bfile {LDSC_DATA_DIR}/1000G_EUR_Phase3_plink/1000G.EUR.QC.{chrom}",
            "--ld-wind-cm 1",
            f"--annot {out_dir}/{chrom}.annot.gz",
            f"--out {out_dir}/{chrom}",
            f"--print-snps {LDSC_DATA_DIR}/listHM3.txt",
        ]
        subprocess.check_call(" ".join(cmds), shell=True)

In [9]:
df_todo_params = df_params[
    ~df_params.apply(lambda x: os.path.exists(x.out_dir + "/22.l2.M"), axis=1)
].reset_index(drop=True)

In [10]:
executor.update_parameters(
    time_min=260,
    memory_g=20,
    #     queue="highp",
    setup=[
        "export PATH=~/project-pasaniuc/software/miniconda3/bin:$PATH",
        "export PATH=~/project-pasaniuc/software/bin:$PATH",
        "export PYTHONNOUSERSITE=True",
    ],
)

jobs2 = executor.map_array(
    make_ldscore,
    df_todo_params.out_dir,
)